In [1]:
import sys 
import os
import pathlib
import click
import hydra
import torch
import dill
import wandb
import json
from diffusion_policy.workspace.base_workspace import BaseWorkspace
from diffusion_policy.env_runner.robomimic_image_runner import RobomimicImageRunner

 
import numpy as np 
import collections 
import tqdm
import h5py
import math 
import wandb.sdk.data_types.video as wv
from diffusion_policy.gym_util.async_vector_env import AsyncVectorEnv
from diffusion_policy.gym_util.sync_vector_env import SyncVectorEnv
from diffusion_policy.gym_util.multistep_wrapper import MultiStepWrapper
from diffusion_policy.gym_util.video_recording_wrapper import VideoRecordingWrapper, VideoRecorder
from diffusion_policy.model.common.rotation_transformer import RotationTransformer

from diffusion_policy.policy.base_image_policy import BaseImagePolicy
from diffusion_policy.common.pytorch_util import dict_apply
from diffusion_policy.env_runner.base_image_runner import BaseImageRunner
from diffusion_policy.env.robomimic.robomimic_image_wrapper import RobomimicImageWrapper
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils 
from diffusion_policy.workspace.train_diffusion_unet_hybrid_workspace import TrainDiffusionUnetHybridWorkspace
from collections import defaultdict, deque

/home/ns1254/miniforge3/envs/robodiff/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/home/ns1254/miniforge3/envs/robodiff/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cuda:0"
device 

device(type='cuda')

In [3]:
checkpoint = "data/outputs/2024.11.03/01.29.56_train_diffusion_unet_hybrid_lift_image/checkpoints/epoch=0550-test_mean_score=0.940.ckpt"
dataset_path="/home/ns1254/data_robomimic/lift/ph/image_v141_f40_abs.hdf5"

In [4]:
checkpoint = "/home/ns1254/diffusion_policy/data/outputs/2024.11.03/12.20.14_train_diffusion_unet_hybrid_can_image/checkpoints/epoch=0450-test_mean_score=0.360.ckpt"
dataset_path="/home/ns1254/data_robomimic/nn/can_image_v141_20p_train_abs.hdf5"

In [5]:
payload = torch.load(open(checkpoint, 'rb'), pickle_module=dill)
cfg = payload['cfg'] 
workspace = TrainDiffusionUnetHybridWorkspace(cfg, output_dir=None) 
workspace.load_payload(payload, exclude_keys=None, include_keys=None)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_eef_quat', 'robot0_gripper_qpos', 'robot0_eef_pos']
using obs modality: rgb with keys: ['robot0_eye_in_hand_image', 'agentview_image']
using obs modality: depth with keys: []
using obs modality: scan with keys: []


/home/ns1254/miniforge3/envs/robodiff/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/ns1254/miniforge3/envs/robodiff/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Diffusion params: 2.556120e+08
Vision params: 2.239418e+07


In [6]:
# get policy from workspace
policy = workspace.model
if cfg.training.use_ema:
    policy = workspace.ema_model

device = torch.device(device)
policy.to(device)
policy.eval()
pass 

In [ ]:
# policy

In [7]:
def create_env(env_meta, shape_meta, enable_render=True):
    modality_mapping = collections.defaultdict(list)
    for key, attr in shape_meta['obs'].items():
        modality_mapping[attr.get('type', 'low_dim')].append(key)
    ObsUtils.initialize_obs_modality_mapping_from_dict(modality_mapping)

    env = EnvUtils.create_env_from_metadata(
        env_meta=env_meta,
        render=False, 
        render_offscreen=enable_render,
        use_image_obs=enable_render, 
    )
    return env

In [8]:
abs_action=True

crf=22
fps=10
n_obs_steps=2
max_steps=500
n_action_steps=8
render_obs_key="agentview_image"

robosuite_fps = 20
steps_per_render = max(robosuite_fps // fps, 1)

In [9]:
env_meta = FileUtils.get_env_metadata_from_dataset(dataset_path)
# disable object state observation
env_meta['env_kwargs']['use_object_obs'] = False

rotation_transformer = None
if abs_action:
    env_meta['env_kwargs']['controller_configs']['control_delta'] = False
    rotation_transformer = RotationTransformer('axis_angle', 'rotation_6d')

shape_meta={'action': {'shape': [10]}, 'obs': {'agentview_image': {'shape': [3, 84, 84], 'type': 'rgb'}, 'robot0_eef_pos': {'shape': [3]}, 'robot0_eef_quat': {'shape': [4]}, 'robot0_eye_in_hand_image': {'shape': [3, 84, 84], 'type': 'rgb'}, 'robot0_gripper_qpos': {'shape': [2]}}}

robomimic_env = create_env(
    env_meta=env_meta, 
    shape_meta=shape_meta
)

Created environment with name PickPlaceCan
Action size is 7


In [10]:
def undo_transform_action( action):
    raw_shape = action.shape
    if raw_shape[-1] == 20:
        # dual arm
        action = action.reshape(-1,2,10)

    d_rot = action.shape[-1] - 4
    pos = action[...,:3]
    rot = action[...,3:3+d_rot]
    gripper = action[...,[-1]]
    rot = rotation_transformer.inverse(rot)
    uaction = np.concatenate([
        pos, rot, gripper
    ], axis=-1)

    if raw_shape[-1] == 20:
        # dual arm
        uaction = uaction.reshape(*raw_shape[:-1], 14)

    return uaction


In [13]:
keys_select = ['agentview_image', 'robot0_eef_pos', 'robot0_eef_quat', 'robot0_eye_in_hand_image', 'robot0_gripper_qpos']

imgs=[]

env=robomimic_env

framestacker=FrameStackForTrans(n_obs_steps)
obs = env.reset()
policy.reset()
obs=framestacker.reset(obs)

In [14]:
np_obs_dict = {key:obs[key][None,:]   for key in keys_select} 
obs_dict = dict_apply(np_obs_dict, lambda x: torch.from_numpy(x).to(device=device))

with torch.no_grad():
    action_dict = policy.predict_action(obs_dict) 
np_action_dict = dict_apply(action_dict, lambda x: x.detach().to('cpu').numpy())

# step env
env_action =  np_action_dict['action']
env_action =  undo_transform_action(env_action)  
env_action=env_action.squeeze()  

/home/ns1254/miniforge3/envs/robodiff/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


In [15]:
env_action =  np_action_dict['action']
env_action.shape

(1, 8, 10)

In [11]:
class FrameStackForTrans:
    def __init__(self, num_frames):
        self.num_frames = num_frames
        self.obs_history = {}
        
    def reset(self, init_obs):
        """ 
        init_obs: dict of initial observation at the start of the episode
        return stacked obs by repeating the first observation num_frames times
        """
        
        self.obs_history = {}
        for k in init_obs:
            self.obs_history[k] = deque([init_obs[k][None] for _ in range(self.num_frames)], maxlen=self.num_frames,)
        obs = { k : np.concatenate(self.obs_history[k], axis=0) for k in self.obs_history }
        return obs 

    def add_new_obs(self, new_obs):
        """ 
        new_obs: dict of new observation at current timestep
        return stacked obs
        """
        for k in new_obs:
            if 'timesteps' in k or 'actions' in k: continue
            self.obs_history[k].append(new_obs[k][None])

        obs= { k : np.concatenate(self.obs_history[k], axis=0) for k in self.obs_history }
        return obs 

In [11]:
def rollout_diffusion(env, policy, n_obs_steps, n_action_steps, max_steps, return_imgs=False):
    keys_select = ['agentview_image', 'robot0_eef_pos', 'robot0_eef_quat', 'robot0_eye_in_hand_image', 'robot0_gripper_qpos']

    imgs=[]

    framestacker=FrameStackForTrans(n_obs_steps)
    obs = env.reset()
    policy.reset()
    obs=framestacker.reset(obs)
    done = False
    success = False
    step =0
    while not done:
        np_obs_dict = {key:obs[key][None,:]   for key in keys_select} 
        obs_dict = dict_apply(np_obs_dict, lambda x: torch.from_numpy(x).to(device=device))
 
        with torch.no_grad():
            action_dict = policy.predict_action(obs_dict) 
        np_action_dict = dict_apply(action_dict, lambda x: x.detach().to('cpu').numpy())
    
        # step env
        env_action =  np_action_dict['action']
        env_action =  undo_transform_action(env_action)  
        env_action=env_action.squeeze()  

        #receding horizon control
        for act in env_action:  
            if return_imgs:
                img = env.render(mode="rgb_array", height=512, width=512, camera_name="agentview")
                imgs.append(img)

            next_obs, reward, done, info = env.step(act)
            success = env.is_success()["task"] 
            step += 1
            if step == max_steps:
                done = True
                break
            
            obs=framestacker.add_new_obs(next_obs)
            if done or success:
                # print(f'---------------------done={done} -- success={success}-------------- ')
                done = True
                break   

        if done:
            break 
    return success, imgs 

In [12]:
success, imgs = rollout_diffusion(robomimic_env, policy, n_obs_steps=2, n_action_steps=8, max_steps=400, return_imgs=True)
success, len(imgs)

/home/ns1254/miniforge3/envs/robodiff/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


(True, 149)

In [14]:
successs=[]
for i in tqdm.tqdm(range(10)):
    success, imgs = rollout_diffusion(robomimic_env, policy, n_obs_steps=2, n_action_steps=8, max_steps=400, return_imgs=True)
    successs.append(success)
    print(f"i={i} success={success}")

mean_success = np.mean(successs)
print('mean success:', mean_success)

 10%|█         | 1/10 [01:49<16:23, 109.23s/it]

i=0 success=False


 20%|██        | 2/10 [03:39<14:38, 109.80s/it]

i=1 success=False


 30%|███       | 3/10 [05:29<12:49, 109.94s/it]

...

i=7 success=True


 90%|█████████ | 9/10 [12:31<01:03, 63.70s/it]

i=8 success=True


100%|██████████| 10/10 [13:19<00:00, 79.98s/it]

i=9 success=True
mean success: 0.4


In [13]:
import imageio
# imgs2=[np.transpose(img, (1,2,0)) for img in imgs]
imageio.mimsave("eval_env3.gif", imgs , duration=0.1)